# Using Clarify to Detect Pre-Training Bias

## Environment Setup

- Image: Data Science
- Kernel: Python 3
- Instance type: ml.t3.medium

## Background

This notebook uses SageMaker Clarify to detect bias in a dataset for home loans.  The dataset, *loan_data.csv*, contains information about customers who applied for a home loan, and whether or not they were approved.  We also use SageMaker Experiments so we can view the Bias Report directly from the Experiments UI.

The dataset was adapted from [Kaggle](https://www.kaggle.com/datasets/devzohaib/eligibility-prediction-for-loan).

## Initialize Environment and Variables

In [ ]:
# Install sagemaker-experiments
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [ ]:
# Import libraries
import boto3
import pandas as pd
import os
from time import sleep, gmtime, strftime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import CSVSerializer
from sagemaker.inputs import TrainingInput

from sagemaker import clarify

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from botocore.exceptions import ClientError

# Get the SageMaker session and the execution role from the SageMaker domain
sess = sagemaker.Session()
role = get_execution_role()

bucket = '<name-of-your-folder>' # Update with the name of a bucket that is already created in S3
prefix = 'demo' # The name of the folder that will be created in the S3 bucket

---
## Data

For this lesson, the data in *loan_data.csv* has been cleaned.  We'll load it into a dataframe, parsing out the target attribute ("Approved"), then take the local file and upload it to S3 so SageMaker can use it.

In [ ]:
# Read data into a dataframe
data_path = 'loan_data.csv'

# Parse out the target attribute ("Approved")
attributes = ['loan_id', 'gender', 'married', 'dependents', 'education', 'self_employed', 'applicant_income', 'coapplicant_income', 'loan_amount', 'term', 'credit_history', 'property_area']
target_attribute = ['approved']
col_names = attributes + target_attribute

df = pd.read_csv(data_path, delimiter=',', index_col=None)
df = df[col_names]

# Print the first five rows
df.head()

In [ ]:
# Upload the file to the S3 bucket defined above
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'loan_data.csv')).upload_file('loan_data.csv')

## Experiments

In this section, we set up our experiment and trials.  Once they're set up, we can hook into them when we start training the model.

In [ ]:
# Create an experiment
create_date = strftime("%Y-%m-%d-%H-%M-%S")
experiment_name = 'loan-approvals-experiment'
experiment_description = 'Analyzing bias in loan approval dataset using SageMaker Clarify.'

# Use a try-block so we can re-use an existing experiment rather than creating a new one each time
try:
    experiment = Experiment.create(experiment_name=experiment_name.format(create_date), 
                                   description=experiment_description)
except ClientError as e:
    print(f'{experiment_name} already exists and will be reused.')

In [ ]:
# Create a trial for the experiment
trial_name = "loan-approvals-trial-1"

demo_trial = Trial.create(trial_name = trial_name.format(create_date),
                          experiment_name = experiment_name)

# Set up experiment_config
experiment_config={'ExperimentName': experiment_name,
                   'TrialName': trial_name,
                   'TrialComponentDisplayName': 'Pretraining-BiasAnalysis'}

## Clarify

In this section, we implement the Clarify code to detect bias in our dataset.  It starts with a processor for the job, then we define various configuration parameters.  

In [ ]:
# Define the processor for the job
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge', 
    sagemaker_session=sess,
    job_name_prefix='clarify-pre-training-bias-detection-job'
)

# Specify the path where the bias report will be saved once complete
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)

# Specify the S3 path to our input data
s3_data_input_path='s3://{}/{}'.format(bucket, prefix)

# Specify inputs, outputs, columns and target names
bias_data_config = clarify.DataConfig(
    s3_data_input_path=s3_data_input_path,
    s3_output_path=bias_report_output_path,
    label='approved',
    headers=df.columns.to_list(),
    dataset_type='text/csv',
)

# Specify the configuration of the bias detection job
# For facet_name, we include two sensitive features we want to check for bias: gender and self-employed
# For facet_values_or_threshold, we input the values of potentially disadvantaged groups (gender of 0 = female; self-employed of 1 = self-employed)
bias_config = clarify.BiasConfig(
    label_values_or_threshold=['Y'], # The value that indicates someone received a home loan
    facet_name=['gender', 'self_employed'],
    facet_values_or_threshold=[[0], [1]]
)

In [ ]:
# Run the bias detection job
clarify_processor.run_pre_training_bias(
    data_config=bias_data_config,
    data_bias_config=bias_config,
    experiment_config=experiment_config,
    logs=False,
)

## Cleaning Up Experiments

In this section, we iterate through our experiments and delete them (this cannot currently be done through the SageMaker UI).

In [ ]:
# Function to iterate through an experiment to delete its trials, then delete the experiment itself
def cleanup_sme_sdk(demo_experiment):
    for trial_summary in demo_experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # Comment out to keep trial components
                tc.delete()
            except:
                # Trial component is associated with another trial
                continue
            # To prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = demo_experiment.experiment_name
    demo_experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
# Call the function above to delete an experiment and its trials
# Fill in your experiment name (not the display name)
experiment_to_cleanup = Experiment.load(experiment_name='loan-approvals-experiment')

cleanup_sme_sdk(experiment_to_cleanup)